In [15]:
import pandas as pd
import numpy as np
import datetime as dt
from w3lib.html import remove_tags

In [16]:
import os
while not os.getcwd().endswith("Data Cleaning"):
    os.chdir("..")

os.getcwd()

'c:\\Users\\gilnr\\OneDrive - NOVASBE\\Work Project\\Thesis - Code\\Data Cleaning'

In [17]:
from pathlib import Path, PureWindowsPath

main_folder = PureWindowsPath("c:\\Users\\gilnr\\OneDrive - NOVASBE\\Work Project\\Thesis - Code")
MAIN_FOLDER = Path(main_folder)
DATA_FOLDER = MAIN_FOLDER / "Data"
DATA_FOLDER

WindowsPath('c:/Users/gilnr/OneDrive - NOVASBE/Work Project/Thesis - Code/Data')

# Load all datasets 

In [12]:
bons_empregos = pd.read_json(DATA_FOLDER / 'bons_empregos_jobs.json')
career_jet = pd.read_json(DATA_FOLDER / 'career_jet_api.json')
carga_de_trabalhos = pd.read_json(DATA_FOLDER / 'CargaDeTrabalhos.json', lines=True)
emprego_xl = pd.read_json(DATA_FOLDER / 'emprego_xl_jobs.json')
emprego_org = pd.read_json(DATA_FOLDER / 'EmpregoOrg.json', lines=True)
itjobs = pd.read_json(DATA_FOLDER / 'itjobs_api.json')
jooble = pd.read_json(DATA_FOLDER / 'jooble_api.json')
landing_jobs = pd.read_json(DATA_FOLDER / 'landingjobs_api.json')
net_empregos = pd.read_json(DATA_FOLDER / 'net_empregos.json')

# Data Cleaning
- For each website there are specific categories that we need to attend. Some require filtering for job location, others cleaning the job description, and the post date.

## Drop Job Vacancies
What makes a unique job vacancy?
- For our analysis it will be: [job_title, job_description, company, job_location]

# General Functions

In [13]:
def description():
    """Clean job description of:
    Weird characters (\n, \r, \r\n,-, etc...)
    """
    pass

In [14]:
def copy_df(dataframe):
   return dataframe.copy()

def replacenan(dataframe):
    dataframe.replace('nan', np.nan, inplace=True)
    return dataframe
    
def dropNullJobs(dataframe):
    """
    Drop null values that make an online job vacancy unusable for analysis.
    The subset to drop is: ['post_date', 'job_title', 'job_description']
    """
    dataframe.dropna(subset=['post_date', 'job_title', 'job_description'], inplace=True)
    return dataframe

# Convert Scrape date to datetime
def toDatetime(dataFrame, columns_list):
    for i in columns_list:
        dataFrame[i] = pd.to_datetime(dataFrame[i])
    return dataFrame

# remove duplicates
def removeDupes(dataframe, subset=['job_title', 'job_description', 'company', 'job_location']):
    dataframe.sort_values(by='post_date').drop_duplicates(subset=subset, keep='last', inplace=True)
    return dataframe

# convert portuguese months to numbers
def longToShortDate(x, sep):
    months = ['janeiro', 'fevereiro','março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
    months_dic = {value:idx+1 for idx, value in enumerate(months)}
    date = [i.strip() for i in x.split(sep)]
    return f'{date[0]}/{months_dic[date[1]]}/{date[2]}'

# convert to datetime object
def convertToDatetime(dataframe, function, sep=' '):
    # Remove comma from date
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: str(x).lower().replace(',',''))
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: dt.datetime.strptime(function(x, sep), "%d/%m/%Y"))
    return dataframe

def listToRows(dataframe, column):
    return dataframe.explode(column)

def removeTags(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(remove_tags)
    return dataframe

def postDatePreprocess(dataframe, sep=" "):
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: x.split(sep)[0]) 
    return dataframe

# Description
def clean_text(text):
    to_replace = ['\r', '\n', '•']
    replace = ['', '', '\n']

    for idx, val in enumerate(to_replace):
        text = text.replace(val, replace[idx])
    text = text.strip()
    return text

def cleanDescription(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(lambda x: clean_text(x))
    return dataframe

## Bons Empregos
- Specific functions:
    - `getPortugalLocation`

In [18]:
def getPortugalLocation(dataframe):
    # Get only job offers in Portugal
    dataframe = dataframe.loc[dataframe['job_location'] != 'Estrangeiro'].copy()
    return dataframe

In [19]:
bons_empregos

,job_location,job_category,job_description,job_title,post_date,scrape_date,job_href,company
0,Estrangeiro,Indústria e Produção,"A Projemato, Lda., está a recrutar ELETRICISTA...",ELETRICISTAS (m/f) para a Suécia,"4 Outubro, 2021",05/10/2021,https://www.bonsempregos.com/oferta-emprego/el...,NaN
1,Estrangeiro,Indústria e Produção,"A Projemato, Lda., está a recrutar ELETRICISTA...",Eletricista (M/F) para Reino Unido,"4 Outubro, 2021",05/10/2021,https://www.bonsempregos.com/oferta-emprego/el...,NaN
2,Estrangeiro,Indústria e Produção,"Dellent is a Portuguese consulting, technology...",Product Designer (UX/UI Designer),"1 Outubro, 2021",05/10/2021,https://www.bonsempregos.com/oferta-emprego/pr...,Dellent
3,Estrangeiro,Indústria e Produção,A KnowledgeWorks é uma Software House com vári...,Call Center - Telecomunicações - Lisboa,"4 Outubro, 2021",05/10/2021,https://www.bonsempregos.com/oferta-emprego/ca...,NaN
4,Estrangeiro,Indústria e Produção,Queres fazer parte de uma empresa de referênci...,Gestor/a de Cliente,"4 Outubro, 2021",05/10/2021,https://www.bonsempregos.com/oferta-emprego/ge...,NaN
...,...,...,...,...,...,...,...,...
2571,Porto,Outros,A RE/MAX Grupo Negócios está a contratar para ...,Gestor de Equipa (m/f),"6 Outubro, 2020",05/10/2021,https://www.bonsempregos.com/oferta-emprego/ge...,Grupo
2572,Porto,Outros,"O Grupo Remax Negócios, presente no mercado im...",Gestor de Clientes (m/f),"6 Outubro, 2020",05/10/2021,https://www.bonsempregos.com/oferta-emprego/ge...,Grupo
2573,Porto,Outros,A Remax foi escolhida pela revista Exame a Mel...,Consultor Imobiliário / Gestor de Equipa (M/F),"6 Outubro, 2020",05/10/2021,https://www.bonsempregos.com/oferta-emprego/co...,NaN
2574,Porto,Outros,Operador de ETAR (M/F) - Braga\nO grupo dst de...,Operador de ETAR (M/F) - Braga,"6 Outubro, 2020",05/10/2021,https://www.bonsempregos.com/oferta-emprego/op...,grupo


In [20]:
bons_empregos_clean = (bons_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date']).
                    pipe(getPortugalLocation).
                    pipe(convertToDatetime, longToShortDate).
                    pipe(removeDupes)
)

print(f'Previous shape: {bons_empregos.shape}\nCurrent shape:{bons_empregos_clean.shape}')

Previous shape: (2576, 8)
Current shape:(2357, 8)


In [21]:
bons_empregos_clean['post_year'] = bons_empregos_clean['post_date'].dt.year
bons_empregos_clean['post_month'] = bons_empregos_clean['post_date'].dt.month
pd.DataFrame(bons_empregos_clean.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

job_title
post_year post_month           
2021      10                 10
          9                 227
          8                  82
          7                 244
          6                 246
          5                 245
          4                 248
          3                 229
          2                 174
          1                 159
2020      12                117
          11                172
          10                204

## Career Jet

In [22]:
# convert job location to list
career_jet['job_location'] = career_jet['job_location'].apply(lambda x: x.split(','))


In [23]:
career_jet_clean = (career_jet.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(listToRows, 'job_location').
                    pipe(removeDupes)
)
print(f'Previous shape: {career_jet.shape}\nCurrent shape:{career_jet_clean.shape}')

Previous shape: (6534, 9)
Current shape:(10613, 9)


In [24]:
career_jet_clean['post_year'] = career_jet_clean['post_date'].dt.year
career_jet_clean['post_month'] = career_jet_clean['post_date'].dt.month
pd.DataFrame(career_jet_clean.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

,,job_title
post_year,post_month,
2021,10,10613


In [25]:
assert career_jet_clean.post_date.dtypes == career_jet_clean.scrape_date.dtypes

## Carga de Trabalhos

In [26]:
carga_de_trabalhos_clean = (carga_de_trabalhos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date']).
                    pipe(convertToDatetime, longToShortDate, '/').
                    pipe(removeDupes)
)

print(f'Previous shape: {carga_de_trabalhos.shape}\nCurrent shape:{carga_de_trabalhos_clean.shape}')

Previous shape: (240, 7)
Current shape:(240, 7)


In [27]:
carga_de_trabalhos_clean['post_year'] = carga_de_trabalhos_clean['post_date'].dt.year
carga_de_trabalhos_clean['post_month'] = carga_de_trabalhos_clean['post_date'].dt.month
pd.DataFrame(carga_de_trabalhos_clean.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

job_title
post_year post_month           
2021      10                102
          9                 108
          8                  30

## Emprego XL

In [28]:
# emprego_xl_clean

## Emprego.Org `SCRAPE AGAIN WITH SCRAPY FOR THE CORRECT FIELDS`

In [54]:
emprego_org

,job_description,job_title,post_date,scrape_date,job_location,job_href,salary,company
0,Abertura Nova Filial- Porto/ Gondomar:,Somos uma multinacional de comércio e serviços...,2021-10-25 / Viewed 30 times,25/10/2021,Porto Porto,https://empregos.org/view.php?job_id=2796890&t...,12000 EUR,NaN
1,Consultor Imobiliário - Porto:Maxgroup,‼ VAGAS NO PORTO ‼\r\nGostava de obter rendime...,2021-10-25 / Viewed 76 times,25/10/2021,Porto,https://empregos.org/view.php?job_id=2796724&t...,25.000 Ano EUR,Maxgroup
2,Novos Talentos – MaisConsultores #PrestigeSul,Gostava de poder mudar a sua vida? De poder te...,2021-10-25 / Viewed 119 times,25/10/2021,Setúbal,https://empregos.org/view.php?job_id=2796720&t...,1250 EUR,NaN
3,Entrada imediata- Full Time- Trofa:Sumptuoso C...,ADMISSÃO IMEDIATA - FULL TIME\r\n\r\nÁreas:\r\...,2021-10-25 / Viewed 126 times,25/10/2021,Maia Porto,https://empregos.org/view.php?job_id=2796725&t...,10000 EUR,Sumptuoso Crescer unip. lda
4,1º Emprego e Desempregados- Porto:,Somos uma multinacional de comércio e serviços...,2021-10-25 / Viewed 267 times,25/10/2021,Porto Porto,https://empregos.org/view.php?job_id=2796714&t...,12000 EUR,NaN
...,...,...,...,...,...,...,...,...
226275,Oferta de emprego: Production Engineer (m/f) -...,Oferta:,2021-10-26 / Viewed 1609 times,26/10/2021,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790328&t...,Não especificado,ENERCON Portugal
226276,Oferta: Maintenance Engineer - Mechanical (m/f...,Anúncio de emprego:,2021-10-26 / Viewed 1593 times,26/10/2021,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790330&t...,Não especificado,ENERCON Portugal
226277,Anúncio de emprego: DIRETOR DE OBRA (M/F) - Zo...,NaN,2021-10-26 / Viewed 1615 times,26/10/2021,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790329&t...,Não especificado,Baltor Steel
226278,Oferta: Operador /a de Produção - Setor Automó...,NaN,2021-10-26 / Viewed 1590 times,26/10/2021,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790331&t...,Não especificado,Eurofirms


In [29]:
# emprego_org_clean = (emprego_org.
#                     pipe(copy_df).
#                     pipe(replacenan).
#                     pipe(dropNullJobs).
#                     pipe(postDatePreprocess, '/').
#                     pipe(toDatetime, ['scrape_date', 'post_date']).
#                     pipe(removeDupes)
# )

## ITJOBS

In [30]:
itjobs_clean = (itjobs.
                    pipe(copy_df).
                    pipe(listToRows, 'job_location').
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {itjobs.shape}\nCurrent shape:{itjobs_clean.shape}')

Previous shape: (3890, 9)
Current shape:(5076, 9)


In [31]:
itjobs_clean['post_year'] = itjobs_clean['post_date'].dt.year
itjobs_clean['post_month'] = itjobs_clean['post_date'].dt.month
pd.DataFrame(itjobs_clean.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

job_title
post_year post_month           
2021      10               2868
          9                2208

## Jooble

In [32]:
jooble_clean = (jooble.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeTags, ['job_title']).
                    pipe(removeDupes)
)

print(f'Previous shape: {jooble.shape}\nCurrent shape:{jooble_clean.shape}')

Previous shape: (1093, 9)
Current shape:(1093, 9)


In [33]:
jooble_clean['post_year'] = jooble_clean['post_date'].dt.year
jooble_clean['post_month'] = jooble_clean['post_date'].dt.month
pd.DataFrame(jooble_clean.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

,,job_title
post_year,post_month,
2021,10,1093


## Landing Jobs IT

In [34]:
landing_jobs_clean = (landing_jobs.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(postDatePreprocess, 'T').
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    # pipe(removeTags, 'job_title').
                    pipe(removeDupes)
)

print(f'Previous shape: {landing_jobs.shape}\nCurrent shape:{landing_jobs_clean.shape}')

Previous shape: (717, 9)
Current shape:(717, 9)


In [35]:
landing_jobs_clean['post_year'] = landing_jobs_clean['post_date'].dt.year
landing_jobs_clean['post_month'] = landing_jobs_clean['post_date'].dt.month
pd.DataFrame(landing_jobs_clean.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

job_title
post_year post_month           
2021      10                  1
          9                  17
          8                   2
          7                   5
          6                  56
          5                 525
          4                  19
          3                  22
          2                  20
          1                   5
2020      12                 16
          11                  4
          10                  8
          9                   1
          8                   4
          7                   3
          6                   1
          5                   2
          4                   1
          3                   4
2018      11                  1

## Net Empregos

In [36]:
pd.to_datetime(net_empregos.post_date.unique())
# net_empregos_clean.post_date.unique()
def invertDate(x):
    if type(x) == float:
        return np.nan
    date = x.split('-')
    return date[2].strip()+'-'+date[1].strip()+'-'+date[0].strip()
    
net_empregos['post_date'] = net_empregos['post_date'].apply(lambda x: invertDate(x))
net_empregos.head()

,job_description,job_title,post_date,scrape_date,company,job_location,job_category,job_href
0,"\r\n A Sector Interactivo, ...",Apoio ao cliente (m/f) Vila Nova de Famalicão,2021-10-4,04/10/2021,Sector Interactivo Lda,Braga,Call Center / Help Desk,https://www.net-empregos.com/7114145/apoio-ao-...
1,\r\n A Jobandtalent encontr...,Cozinheiro de 3ª (M/F) Aeroporto Faro,2021-10-4,04/10/2021,JOBANDTALENT Portugal,Faro,Restauração / Bares / Pastelarias,https://www.net-empregos.com/7582409/cozinheir...
2,\r\n Blissa is a Real Estat...,Real Estate Agent,2021-10-4,04/10/2021,Blissa - Real Estate Consulting,Lisboa,Imobiliário,https://www.net-empregos.com/7743402/real-esta...
3,\r\n TÉCNICA(O) DE FARMÁCIA...,Farmácia Branco Lisboa em Caldas da Rainha pro...,2021-10-4,04/10/2021,"Branco Lisboa, Limitada (Farmácia Branco Lisboa)",Leiria,Farmácia / Biotecnologia,https://www.net-empregos.com/7752949/farmacia-...
4,"\r\n Empregado/a balcão, me...","Precisa-se empregado/a para balcão, mesas , Sa...",2021-10-4,04/10/2021,A Primórdio Lda,Lisboa,Restauração / Bares / Pastelarias,https://www.net-empregos.com/7753235/precisa-s...


In [37]:
pd.to_datetime(net_empregos.post_date.unique())

DatetimeIndex(['2021-10-04', '2021-10-05', '2021-10-03', '2021-10-02',
               '2021-10-01', '2021-09-30', '2021-09-29', '2021-09-28',
               '2021-09-27',        'NaT', '2021-09-26', '2021-09-25',
               '2021-09-24', '2021-09-23', '2021-09-22', '2021-09-21',
               '2021-09-20', '2021-09-19', '2021-09-18', '2021-09-17',
               '2021-09-16', '2021-09-15', '2021-09-14', '2021-09-13',
               '2021-09-12', '2021-09-11', '2021-09-10', '2021-09-09',
               '2021-09-08', '2021-09-07', '2021-09-06'],
              dtype='datetime64[ns]', freq=None)

In [38]:
def invertDate(x):
    if type(x) == float:
        return np.nan
    date = x.split('-')
    return date[2].strip()+'-'+date[1].strip()+'-'+date[0].strip()

def pipeInvertDate(dataframe, function=invertDate):
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: function(x))
    return dataframe

In [39]:
net_empregos_clean = (net_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    # pipe(pipeInvertDate).
                    pipe(dropNullJobs).
                    pipe(cleanDescription, ['job_title']).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {net_empregos.shape}\nCurrent shape:{net_empregos_clean.shape}')

Previous shape: (54800, 8)
Current shape:(54791, 8)


In [40]:
net_empregos_clean['post_year'] = net_empregos_clean['post_date'].dt.year
net_empregos_clean['post_month'] = net_empregos_clean['post_date'].dt.month
pd.DataFrame(net_empregos_clean.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

job_title
post_year post_month           
2021      10              21991
          9               32800

# Add Website Column to all dataframes before concat

In [41]:
jobs_dfs = [bons_empregos_clean, career_jet_clean, carga_de_trabalhos_clean, emprego_org_clean, #emprego_xl_clean, emprego_org_clean, 
            itjobs_clean, jooble_clean, landing_jobs_clean, net_empregos_clean]
websites = ['Bons empregos', 'Career Jet', 'Carga de Trabalhos',#'Emprego XL', 'Emprego.org' 
            'ITjobs','Jooble','Landing Jobs','Net-empregos']

# Add column with website name
for idx, value in enumerate(jobs_dfs):
    value['website'] = websites[idx]   

# Concat All dataframes into one for data Deduplication

In [42]:
neworder = ['job_title','job_description','company','job_location','job_category','salary', 'post_date', 'scrape_date','job_href', 'website']

df = pd.concat([i.reindex(columns=neworder) for i in jobs_dfs])

# Validate that the concatenation is happening properly
assert len(df) == sum(len(i) for i in jobs_dfs)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74887 entries, 10 to 54799
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        74887 non-null  object        
 1   job_description  74887 non-null  object        
 2   company          74147 non-null  object        
 3   job_location     74887 non-null  object        
 4   job_category     74647 non-null  object        
 5   salary           12483 non-null  object        
 6   post_date        74887 non-null  datetime64[ns]
 7   scrape_date      74887 non-null  datetime64[ns]
 8   job_href         68001 non-null  object        
 9   website          74887 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 6.3+ MB


## Pipeline

In [44]:
def cleanCompany(dataframe):
    def capitalize(x):
        try:
            return x.capitalize()
        except AttributeError:
            return ''
    dataframe['company'] = dataframe['company'].apply(lambda x: capitalize(x))
    return dataframe

In [45]:
df_clean = (df.
            pipe(copy_df).
            pipe(replacenan).
            pipe(dropNullJobs).
            pipe(cleanCompany).
            pipe(cleanDescription, ['job_title', 'job_description']).
            pipe(removeDupes, ['job_title', 'company', 'job_location'])
)
df_clean.reset_index(drop=True, inplace=True)

print(f'Previous shape: {df.shape}\nCurrent shape:{df_clean.shape}')

Previous shape: (74887, 10)
Current shape:(74887, 10)


In [46]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74887 entries, 0 to 74886
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        74887 non-null  object        
 1   job_description  74887 non-null  object        
 2   company          74887 non-null  object        
 3   job_location     74887 non-null  object        
 4   job_category     74647 non-null  object        
 5   salary           12483 non-null  object        
 6   post_date        74887 non-null  datetime64[ns]
 7   scrape_date      74887 non-null  datetime64[ns]
 8   job_href         68001 non-null  object        
 9   website          74887 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 5.7+ MB


In [47]:
print(f'There is a total of {len(df_clean)} jobs of which {df_clean.job_title.nunique()} have unique titles')

There is a total of 74887 jobs of which 53415 have unique titles


In [48]:
df.describe()

<ipython-input-48-ea8415b8a3ee>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()
<ipython-input-48-ea8415b8a3ee>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,job_title,job_description,company,job_location,job_category,salary,post_date,scrape_date,job_href,website
count,74887,74887,74147,74887,74647,12483,74887,74887,68001,74887
unique,53434,53279,12931,538,68,143,4151,4,58281,7
top,Ajudante de Cozinha,\r\n,,Lisboa,,,2021-10-22 00:00:00,2021-05-10 00:00:00,http://jobviewtrack.com/pt-pt/job-1d12416e4c16...,Net-empregos
freq,401,5254,7497,20559,17499,6030,10233,57130,40,54791
first,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-16 00:00:00,2021-04-10 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-25 00:00:00,2021-10-25 00:00:00,NaN,NaN


In [49]:
df.head()

,job_title,job_description,company,job_location,job_category,salary,post_date,scrape_date,job_href,website
10,Backend Developer (Python),"Na Dellent, ligamos os melhores talentos aos m...",Dellent,Porto,Informática (Programação),NaN,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/ba...,Bons empregos
11,Assistant de gestion Assurance - Français,"Génération, une entreprise innovante, motivant...",GENERATION,Porto,Informática (Programação),NaN,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/as...,Bons empregos
12,Business Intelligence Developer (Tableau),"Dellent Consulting is a Portuguese consulting,...",Dellent,Porto,Informática (Programação),NaN,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/bu...,Bons empregos
13,Senior Python/Angular Developer,A Dellent Consulting é uma consultora especial...,Dellent,Porto,Informática (Programação),NaN,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/se...,Bons empregos
14,Application Maintenance Engineer,"At Dellent Consulting, we know that with the r...",Dellent,Porto,Informática (Programação),NaN,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/ap...,Bons empregos


In [50]:
df_clean.to_json(DATA_FOLDER / 'full_data_clean.json', date_unit='s')
df_clean.to_csv(DATA_FOLDER / 'full_data_clean.csv')
# df_clean.to_json()

In [51]:
df['post_year'] = df['post_date'].dt.year
df['post_month'] = df['post_date'].dt.month

In [52]:
pd.DataFrame(df.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

job_title
post_year post_month           
2021      10              36678
          9               35360
          8                 114
          7                 249
          6                 302
          5                 770
          4                 267
          3                 251
          2                 194
          1                 164
2020      12                133
          11                176
          10                212
          9                   1
          8                   4
          7                   3
          6                   1
          5                   2
          4                   1
          3                   4
2018      11                  1